In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson

import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open('models/rf_home.pkl', 'rb') as f:
    home_md = pickle.load(f)

with open('models/rf_away.pkl', 'rb') as f:
    away_md = pickle.load(f)


teams_data = pd.read_csv("data/new_model.csv")   

In [ ]:
def find_team_features(home, away):
    home_infos = teams_data[(teams_data.home_team == home) | (teams_data.away_team == home)].tail(1)
    if home_infos.home_team.values[0] == home:
        home_goals_scored = home_infos["home_goals_mean"].values[0]
        home_goals_suf = home_infos["home_goals_suf_mean"].values[0]
        home_points_made = home_infos["home_game_points_mean"].values[0]
        home_rank = home_infos["rank_home"].values[0]
    else:
        home_goals_scored = home_infos["away_goals_mean"].values[0]
        home_goals_suf = home_infos["away_goals_suf_mean"].values[0]
        home_points_made = home_infos["away_game_points_mean"].values[0]
        home_rank = home_infos["rank_away"].values[0]
        
    
    away_infos = teams_data[(teams_data.home_team == away) | (teams_data.away_team == away)].tail(1)
    if away_infos.home_team.values[0] == away:
        away_goals_scored = away_infos["home_goals_mean"].values[0]
        away_goals_suf = away_infos["home_goals_suf_mean"].values[0]
        away_points_made = away_infos["home_game_points_mean"].values[0]
        away_rank = away_infos["rank_home"].values[0]
    else:
        away_goals_scored = away_infos["away_goals_mean"].values[0]
        away_goals_suf = away_infos["away_goals_suf_mean"].values[0]
        away_points_made = away_infos["away_game_points_mean"].values[0]
        away_rank = away_infos["rank_away"].values[0]

    #print(home_rank, away_rank)
    rank_dif = home_rank - away_rank
    ht_features = [
        home_goals_scored*rank_dif,
        away_goals_suf*rank_dif,
        home_points_made*rank_dif,
        away_points_made*rank_dif
    ]
    
    at_features = [
        away_goals_scored*rank_dif,
        home_goals_suf*rank_dif,
        home_points_made*rank_dif,
        away_points_made*rank_dif
    ]
    return ht_features, at_features

In [ ]:
g, s = find_team_features("Italy", "Albania")

xg_g = home_md.predict([g])[0]
xg_s = away_md.predict([s])[0]


poisson_probs_g = poisson.pmf(np.arange(0, 10), xg_g)
poisson_probs_s = poisson.pmf(np.arange(0, 10), xg_s)


def plot_poisson(x, poisson_probs, team):
    plt.figure(figsize=(10, 6))
    plt.bar(x, poisson_probs, alpha=0.6, color='blue')
    plt.plot(x, poisson_probs, 'bo', ms=8)
    plt.title(f'Poisson Distribution - {team}')
    plt.xlabel('Number of Events')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.show()

plot_poisson(np.arange(0,10), poisson_probs_g, "Italy")
plot_poisson(np.arange(0,10), poisson_probs_s, "Albania")

res_track = np.zeros((10,10))

for i in range(len(poisson_probs_g)):
    for j in range(len(poisson_probs_s)):
        res_prob = poisson_probs_g[i]*poisson_probs_s[j]
        res_track[j][i] = res_prob

res_df = pd.DataFrame(res_track)

res_df.columns = ["Italy " + str(a) for a in res_df.columns] 
res_df.index = ["Albania " + str(a) for a in res_df.index] 


def find_probs(poisson_home, poisson_away):
    home_probs = []
    away_probs = []
    draw_probs = []
    both_score_probs = []
    over_1_5_probs = []
    under_1_5_probs = []
    over_2_5_probs = []
    under_2_5_probs = []
    
    for i in range(len(poisson_home)):
        for j in range(len(poisson_away)):
            res_prob = poisson_home[i] * poisson_away[j]
            
            if i == j:
                draw_probs.append(res_prob)
            elif i > j:
                home_probs.append(res_prob)
            elif j > i:
                away_probs.append(res_prob)
            
            # Both teams score
            if i > 0 and j > 0:
                both_score_probs.append(res_prob)
            
            # Over and under 1.5 goals
            if (i + j) > 1.5:
                over_1_5_probs.append(res_prob)
            else:
                under_1_5_probs.append(res_prob)
                
            # Over and under 2.5 goals
            if (i + j) > 2.5:
                over_2_5_probs.append(res_prob)
            else:
                under_2_5_probs.append(res_prob)
    
    both_score_prob = sum(both_score_probs)
    over_1_5_prob = sum(over_1_5_probs)
    under_1_5_prob = sum(under_1_5_probs)
    over_2_5_prob = sum(over_2_5_probs)
    under_2_5_prob = sum(under_2_5_probs)

    # max_prob_index = np.unravel_index(np.argmax(res_track), res_track.shape)
    # g1, g2 = max_prob_index

    
    return (sum(home_probs), sum(draw_probs), sum(away_probs), 
            poisson_home.tolist().index(max(poisson_home.tolist())), 
            poisson_away.tolist().index(max(poisson_away.tolist())),
            both_score_prob, over_1_5_prob, under_1_5_prob, 
            over_2_5_prob, under_2_5_prob)